In [1]:
%%writefile Dockerfile

FROM python:3.9-slim-buster

RUN pip install pandas==2.2.2 scikit-learn==1.4.2 prophet==1.1.5

COPY code/preprocess.py /opt/ml/code/preprocess.py
COPY code/train.py /opt/ml/code/train.py

ENV SAGEMAKER_PROGRAM preprocess.py
ENV SAGEMAKER_PROGRAM train.py
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]

Overwriting Dockerfile


In [2]:
import boto3

account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.Session().region_name
ecr_repository = 'demo_prophet'
tag = ':latest'
processing_repository_uri = '{}.dkr.ecr.{}.amazonaws.com/{}'.format(account_id, region, ecr_repository + tag)

In [3]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository .

Sending build context to Docker daemon  174.1kB
Step 1/8 : FROM python:3.9-slim-buster
 ---> c84dbfe3b8de
Step 2/8 : RUN pip install pandas==2.2.2 scikit-learn==1.4.2 prophet==1.1.5
 ---> Using cache
 ---> a46aef910c10
Step 3/8 : COPY code/preprocess.py /opt/ml/code/preprocess.py
 ---> 0085b4d348f2
Step 4/8 : COPY code/train.py /opt/ml/code/train.py
 ---> 561835458669
Step 5/8 : ENV SAGEMAKER_PROGRAM preprocess.py
 ---> Running in 593ec15c6d8b
Removing intermediate container 593ec15c6d8b
 ---> 2ecdd4b67660
Step 6/8 : ENV SAGEMAKER_PROGRAM train.py
 ---> Running in 248e4c560c67
Removing intermediate container 248e4c560c67
 ---> 6073a3449ccf
Step 7/8 : ENV PYTHONUNBUFFERED=TRUE
 ---> Running in be4ea78b51d1
Removing intermediate container be4ea78b51d1
 ---> e3054966626c
Step 8/8 : ENTRYPOINT ["python3"]
 ---> Running in 6bd41e4ceddb
Removing intermediate container 6bd41e4ceddb
 ---> a2e977f88ecf
Successfully built a2e977f88ecf
Successfully tagged demo_prophet:latest


In [4]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [5]:
!aws ecr create-repository --repository-name $ecr_repository


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'demo_prophet' already exists in the registry with id '654654222480'


In [6]:
!docker tag {ecr_repository + tag} $processing_repository_uri

In [7]:
!docker push $processing_repository_uri

The push refers to repository [654654222480.dkr.ecr.us-east-1.amazonaws.com/demo_prophet]

d79d1e09: Preparing 
4b1d6eb1: Preparing 
d0cba51b: Preparing 
a27560c1: Preparing 
037e08b3: Preparing 
eede8d6e: Preparing 
55769c5e: Preparing 
d79d1e09: Pushed lready exists latest: digest: sha256:33671d74386451fe93c1978fd3b644f8cd1e35bf72e0534cbc281b8645046c49 size: 1997
